In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt

%matplotlib notebook

paths = [
    r"C:\Users\Administrator\Code\qcodes_jens",
    r"C:\Users\Administrator\Code\pulsar",
]

for p in paths:
    if p not in sys.path:
        sys.path.append(p)

import qcodes as qc
from pulsar import pulse, element
from pulsar.util import view

from importlib import reload
reload(pulse)
reload(element)
reload(view)

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


<module 'pulsar.util.view' from 'C:\\Users\\Administrator\\Code\\pulsar\\pulsar\\util\\view.py'>

In [2]:
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
awg = Tektronix_AWG5014('awg', address="TCPIP0::169.254.183.196::inst0::INSTR")

Connected to: TEKTRONIX AWG5014C (serial:B051388, firmware:SCPI:99.0 FW:4.6.0.7) in 0.10s


In [3]:
# view.show_element(e)
# view.show_element(f)

In [3]:
def prepare_wfs(elements, channel_map):
    allwfs = []
    allm1s = []
    allm2s = []
    
    chans = []
    for ch in channel_map.keys():
        if channel_map[ch] == 3 * [None]:
            continue
        chans.append(ch)
        
        wfs = []
        m1s = []
        m2s = []
        
        for i, elt in enumerate(elements):
            ts, ws = elt.waveforms()
            fzeros = 0.0 * ts
            izeros = np.zeros(ts.size, dtype=np.int)
        
            wfs.append(ws.pop(channel_map[ch][0], fzeros))
            m1s.append(ws.pop(channel_map[ch][1], izeros))
            m2s.append(ws.pop(channel_map[ch][2], izeros))
            
        allwfs.append(wfs)
        allm1s.append(m1s)
        allm2s.append(m2s)
                    
    return allwfs, allm1s, allm2s

In [ ]:
# w, m1, m2 = prepare_wfs(elements, chan_map)
# w

#For singlge line scan use length = 500e-6, length2 = 2e-3, 120mVpp on AWG, 10MS/s.  This gives signal from blockade oscillations ~ 340Hz in FFT
#For 2D scan use length = 90e-6, 50mVpp on AWG, 10MS/s

In [5]:
chan_map = {
    1 : [None, None, None],
    2 : ['G', 'trig', None],
    3 : [None] * 3,
    4 : [None] * 3,
}

elements = []
npts = 200
npts2 = 100
length = 90e-6
length2 = 2e-3/npts2
amps = np.linspace(0, 1, npts)
amps2 = np.linspace(1, 0, npts2)

e = element.Element('elt1', clock=1e7, min_samples=2500)
e.define_channel('G', delay=0, type='analog', high=1, low=-1, offset=0)
e.define_channel('trig', type='marker', high=1, low=0, delay=0)

name = None
for i, amp in enumerate(amps):
    name = e.add(
        pulse.cp(pulse.SquarePulse('G', 'gate'), 
                 amplitude=amp, length=length),
        refpulse=name, refpoint='end',
    )
    
    _ = e.add(
        pulse.cp(pulse.SquarePulse('trig', 'trigger'),
                 amplitude=1, length=1e-6),
        refpulse=name, refpoint='start', start=1e-6,
    )
    
for i, amp in enumerate(amps2):
    name = e.add(
        pulse.cp(pulse.SquarePulse('G', 'gate'),
                 amplitude=amp, length=length2),
        refpulse=name, refpoint='end',
    )

elements.append(e)

nelts = len(elements)
w, m1, m2 = prepare_wfs(elements, chan_map)
awg.make_send_and_load_awg_file(w, m1, m2, nelts * [1], nelts * [0], (nelts-1)*[0] + [1], nelts*[0], nelts*[2], 
                                "ramp.awg")

In [ ]:
# e.print_overview()
print(e.length())
print(e.samples())

In [ ]:
(500 * 0.2)